In [13]:
# color coding function
def color_code_fn(school_type):
    if school_type[i] == "elementary school":
        color_code = [255,0,125,255]
    elif school_type[i] == "middle school":
        color_code = [255,0,0,255]  
    elif school_type[i] == "high school":
        color_code = [125,0,255,255] 
    else:
        color_code = [125,0,255,255] 
    return color_code


# Today

In [14]:
import datetime
import pandas as pd
import numpy as np
from datascience import *
import random
from arcgis.gis import GIS

now = datetime.datetime.now()

# read in schools table
school_df = pd.read_pickle('geocoding_schools/cleaned_school_list.pk1')


#from arcgis.geocoding import geocoding
my_gis = GIS()
seattle = my_gis.map("Seattle, WA")

geocodes = school_df['geocode']
name = school_df['School']

i = 0
for school in geocodes:
    try:
        seattle.draw(school['location'], {"title" : name[i], "content" : school['address']},
                     {"type": "esriSMS","style": "esriSMSCircle","color": [0,0,0,125],"size": 4,
                      "angle": 0,"xoffset": 0,"yoffset": 0,
                      "outline": {"color": [0,0,0,255],"width": 0}})
    except:
        pass
        #print("No address for:", (name[i]))
    i+=1
    
late_bus_df = pd.read_pickle('scraping_bus_data/bus_data.pk1')
late_bus_df1  = late_bus_df.loc[late_bus_df['Day'] == str(7)]#str(now.day)]
late_bus_df2  = late_bus_df1.loc[late_bus_df1['To/From'] == 'to']
late_bus_df3  = late_bus_df2.loc[late_bus_df2['Data Taken Hour'] == 7]
late_schools = school_df.loc[school_df.get('School').isin(late_bus_df3.get('School'))]

# calculate the total bus late time for each school
total_rows = int(late_schools.size/8)

late_time = (total_rows)*[0]
i = 0

# add minute late column to school df
for index_schools, row_schools in late_schools.iterrows():
    for index_buses, row_buses in late_bus_df3.iterrows():
        if row_schools['School'] == row_buses['School']:
            #print(row_schools['School'],row_buses['School'])
            late_time[i] += row_buses['Time']
    i+=1
    #print(late_time)
    
late_schools['Late Time'] = late_time

# map late schools
geocodes = late_schools['geocode']
name = list(late_schools['School'])
late_time = list(late_schools['Late Time'])
school_type = list(late_schools['Type'])

#print(late_time)
i = 0
for school in geocodes:
    max_d = 8
    #if school_type[i] == "elementary school":
    #    color_code = [255,0,125,255]
    #elif school_type[i] == "middle school":
    #    color_code = [255,0,0,255]  
    #elif school_type[i] == "high school":
    #    color_code = [125,0,255,255] 
    #else:
    #    color_code = [125,0,255,255] 
    color_code = color_code_fn(school_type)
    
    d = (max_d*(late_time[i]+30))/max(late_time)
    seattle.draw(school['location'], {"title" : name[i], "content" : "Late: "+str(late_time[i])+" minutes"},
                     {"type": "esriSMS","style": "esriSMSCircle","color": color_code,
                      "size": d,
                      "angle": 0,"xoffset": 0,"yoffset": 0,
                      "outline": {"color": [0,0,0,255],"width": 1}})

    i+=1
       
seattle

               School               Type        Street Address Zip Code  \
5         Nathan Hale        high school    10750 30th Ave NE     98125   
36               Orca                k-8      5215 46th Ave Sæ    98118   
38         Salmon Bay                k-8      1810 NW 65th Stæ    98117   
61        Graham Hill  elementary school     5149 S Graham St     98118   
64          Hawthorne  elementary school      4100 39th Ave S     98118   
71             Lawton  elementary school      4000 27th Ave W     98199   
73             Lowell  elementary school     1058 E Mercer St     98102   
79  Thurgood Marshall  elementary school     2401 S Irving St     98144   
81           McGilvra  elementary school      1617 38th Ave E     98112   
97     Thornton Creek  elementary school  7712 40th Avenue NE     98115   

   option_alt    latitude  longitude  \
5         N/A -122.296121  47.708046   
36     option -122.275402  47.554785   
38     option -122.380510  47.676016   
61        N/A 

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


NameError: name 'school_types' is not defined

# All Days